In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!cp drive/My\ Drive/style.jpg /content/
!cp drive/My\ Drive/style2.jpg /content/
!cp drive/My\ Drive/style3.jpg /content/
!cp drive/My\ Drive/basee.jpg /content/

cp: cannot stat 'drive/My Drive/style.jpg': No such file or directory


In [14]:
!ls

basee.jpg  drive  output.jpg  sample_data  style2.jpg  style3.jpg  style.jpg


In [4]:
import numpy as np
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import cv2 as cv
import time
from scipy.optimize import fmin_l_bfgs_b

Using TensorFlow backend.


# **Pre-process Image**

In [0]:
def build_input_tensor(width, height, content_image, style_image, style_image2=None):
  # Add one more dimension to concatinate the images
  content_array = np.asarray(content_image, dtype='float32')
  content_array = np.expand_dims(content_array, axis=0)

  style_array = np.asarray(style_image, dtype='float32')
  style_array = np.expand_dims(style_array, axis=0)
  
  style_array2 = np.asarray(style_image2, dtype='float32')
  style_array2 = np.expand_dims(style_array2, axis=0)

  # Substract mean RGB values from images to make them ready to VGG16
  content_array[:, :, :, 0] -= 103.939
  content_array[:, :, :, 1] -= 116.779
  content_array[:, :, :, 2] -= 123.68
  content_array = content_array[:, :, :, ::-1]

  style_array[:, :, :, 0] -= 103.939
  style_array[:, :, :, 1] -= 116.779
  style_array[:, :, :, 2] -= 123.68
  style_array = style_array[:, :, :, ::-1]
  
  # Build input tensor with Keras placeholder
  content_image = K.variable(content_array)
  style_image = K.variable(style_array)
  combination_image = K.placeholder((1, height, width, 3))

  if style_image2 is not None:
    style_array2[:, :, :, 0] -= 103.939
    style_array2[:, :, :, 1] -= 116.779
    style_array2[:, :, :, 2] -= 123.68
    style_array2 = style_array2[:, :, :, ::-1]
    style_image2 = K.variable(style_array2)
    input_tensor = K.concatenate([content_image, style_image, combination_image, style_image2], axis=0)
  else:
    input_tensor = K.concatenate([content_image, style_image, combination_image], axis=0)

  return input_tensor, combination_image

# **Content Loss**

In [0]:
def content_loss(content_layer, model, layers, content_weight):
  # Calculate the Content Loss, extract the activations from CNN
  layer_features = layers[content_layer]
  content_image_features = layer_features[0, :, :, :]
  combination_features = layer_features[2, :, :, :]

  # Calculate the OLS error between combined image and content image
  content_loss = K.sum(K.square(combination_features - content_image_features))
  return content_weight * content_loss

# **Style Loss**

In [0]:
def style_loss(style_layers, layers, style_weight, width, height, idx):
  # Layers used to calculate style loss
  loss = K.variable(0.)
  # Sum the losses we get from each layer for the style loss
  for layer_name in style_layers:
      layer_features = layers[layer_name]
      style_features = layer_features[idx, :, :, :]
      combination_features = layer_features[2, :, :, :]
      sl = style_loss_single(style_features, combination_features, width, height)
      loss += (style_weight / len(style_layers)) * sl
  return loss

# Calculate Gram Matrix of given matrix (X*X^T)
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# Style loss calculated from single layer features
def style_loss_single(style, combination, width, height):
    style_gram = gram_matrix(style)
    content_gram = gram_matrix(combination)
    channels = 3
    size = height * width
    return K.sum(K.square(style_gram - content_gram)) / (4. * (channels ** 2) * (size ** 2))

# **Total Variation Loss**

In [0]:
def total_variation_loss(combination_image, total_variation_weight, width, height):
    a = K.square(combination_image[:, :height-1, :width-1, :] - combination_image[:, 1:, :width-1, :])
    b = K.square(combination_image[:, :height-1, :width-1, :] - combination_image[:, :height-1, 1:, :])
    return total_variation_weight * K.sum(K.pow(a + b, 1.25))

# **Optimization**

In [0]:
def minimize_loss(loss, combination_image, width, height):
  # Calculate gradients of the total loss w.r.t. combined image
  gradients = K.gradients(loss, combination_image)

  outputs = [loss]
  outputs += gradients
  f_outputs = K.function([combination_image], outputs)
  
  def eval_loss_and_grads(x):
      x = x.reshape((1, height, width, 3))
      outs = f_outputs([x])
      loss_value = outs[0]
      grad_values = outs[1].flatten().astype('float64')
      return loss_value, grad_values

  class Evaluator(object):

      def __init__(self):
          self.loss_value = None
          self.grads_values = None

      def loss(self, x):
          assert self.loss_value is None
          loss_value, grad_values = eval_loss_and_grads(x)
          self.loss_value = loss_value
          self.grad_values = grad_values
          return self.loss_value

      def grads(self, x):
          assert self.loss_value is not None
          grad_values = np.copy(self.grad_values)
          self.loss_value = None
          self.grad_values = None
          return grad_values

  
  evaluator = Evaluator()

  x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

  iterations = 10

  for i in range(iterations):
      print('Start of iteration', i+1)
      start_time = time.time()
      x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
      print('Current loss value:', min_val)
      end_time = time.time()
      print('Iteration %d completed in %ds\n' % (i+1, end_time - start_time))

  x = x.reshape((height, width, 3))
  x = x[:, :, ::-1]
  x[:, :, 0] += 103
  x[:, :, 1] += 116
  x[:, :, 2] += 123
  x = np.clip(x, 0, 255).astype('uint8')
  return Image.fromarray(x)

# **Style Transfer**

In [0]:
def transfer_style(width, height, style_layers, content_layer, content, style, style2=None, content_weight = 0.025, style_weight = 5.0, total_variation_weight = 1.0):
  # Load content image
  content_image = Image.open(content)
  content_image = content_image.resize((width, height))
   
  # Load style image
  style_image = Image.open(style)
  style_image = style_image.resize((width, height))
  
  # If we have 2 style images
  if style2 is not None:
    # Load style image
    style_image2 = Image.open(style2)
    style_image2 = style_image2.resize((width, height))
    input_tensor, combination_image = build_input_tensor(width, height, content_image, style_image,style_image2)
  else:
    input_tensor, combination_image = build_input_tensor(width, height, content_image, style_image)
    
  # Create instance of pre-trained VGG16 Classifier model, do not include last layer
  model = vgg16.VGG16(input_tensor=input_tensor, weights="imagenet", include_top=False)
  layers = dict([(layer.name, layer.output) for layer in model.layers])
  print(layers)
  
  # Calculate content loss, style loss and total variation loss
  loss = K.variable(0.)
  loss += content_loss(content_layer, model, layers, content_weight)
  loss += style_loss(style_layers, layers, style_weight, width, height,1)
  if style2 is not None:
    loss += style_loss(style_layers, layers, style_weight, width, height,3)
  loss += total_variation_loss(combination_image, total_variation_weight, width, height)
  
  # Minimize total loss by L-BFGS Algorithm
  result_img = minimize_loss(loss, combination_image, width, height)
  
  return result_img

# Test

In [25]:
'''
# Import Base and Style images and resize them into same dimension
content = 'basee.jpg'
style = 'style3.jpg'
style2 = 'style2.jpg'

style_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
content_layer = 'block2_conv2'

result = transfer_style(400, 500, style_layers, content_layer, content, style, content_weight = 0.01, style_weight = 8.0, total_variation_weight = 1.0,)
result.save('output.jpg')
!cp /content/output.jpg drive/My\ Drive/
result
'''

{'input_4': <tf.Tensor 'concat_3:0' shape=(3, 500, 400, 3) dtype=float32>, 'block1_conv1': <tf.Tensor 'block1_conv1_3/Relu:0' shape=(3, 500, 400, 64) dtype=float32>, 'block1_conv2': <tf.Tensor 'block1_conv2_3/Relu:0' shape=(3, 500, 400, 64) dtype=float32>, 'block1_pool': <tf.Tensor 'block1_pool_3/MaxPool:0' shape=(3, 250, 200, 64) dtype=float32>, 'block2_conv1': <tf.Tensor 'block2_conv1_3/Relu:0' shape=(3, 250, 200, 128) dtype=float32>, 'block2_conv2': <tf.Tensor 'block2_conv2_3/Relu:0' shape=(3, 250, 200, 128) dtype=float32>, 'block2_pool': <tf.Tensor 'block2_pool_3/MaxPool:0' shape=(3, 125, 100, 128) dtype=float32>, 'block3_conv1': <tf.Tensor 'block3_conv1_3/Relu:0' shape=(3, 125, 100, 256) dtype=float32>, 'block3_conv2': <tf.Tensor 'block3_conv2_3/Relu:0' shape=(3, 125, 100, 256) dtype=float32>, 'block3_conv3': <tf.Tensor 'block3_conv3_3/Relu:0' shape=(3, 125, 100, 256) dtype=float32>, 'block3_pool': <tf.Tensor 'block3_pool_3/MaxPool:0' shape=(3, 62, 50, 256) dtype=float32>, 'block4